In [1]:
import os
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Activation, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score, classification_report

np.random.seed(seed=17)

### Simulation Subset Selection
- Use the results summary file to select a subset of simulations for model training and testing
    - Randomly select n non-fail cases and n fail cases
    - Use the simulation ids to read in the corresponding simulation data files


In [2]:
# Import simulation results
results = pd.read_csv('processed_results_summary.csv', keep_default_na=False)
results.head()

,Unnamed: 0,Simulation no#,Line,Load,Wind1,Wind2,Wind3,flag,Sequence,n_failures,t_fail,t_first_fail,dt_fail,failed component_type,failed_component_type,cascade,fail
0,0,1,Line 04 - 05,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
1,18,10,Line 26 - 28,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
2,198,100,Line 09 - 39,0.7,0.0,0.0,0.4,0,[],0,,,,,,False,False
3,1998,1000,Line 22 - 23,0.7,0.0,0.8,1.0,0,[],0,,,,,,False,False
4,19998,10000,Line 10 - 11,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False


In [3]:
# List data files and read in the selected simulations
DATA_PATH = '/output/02_preprocessed_numpy'
files = os.listdir(DATA_PATH)

In [4]:
# Get the simulation ids of sims with and without failures
fail_ids = results[results['flag']==1]['Simulation no#'].tolist()
non_fail_ids = results[results['flag']==0]['Simulation no#'].tolist()

# Randomly select fail and non-fail ids
fail_ids = np.random.choice(fail_ids, size=len(fail_ids), replace=False)
non_fail_ids = np.random.choice(non_fail_ids, size=30000, replace=False)

# Get full filepaths
paths_fail, paths_non_fail = [], []
labels_fail, labels_non_fail = [], []

for file in files:
    file_id = int(file.split('.pkl')[0])

    if file_id in fail_ids and len(paths_fail) < 3500:
        paths_fail.append(os.path.join(DATA_PATH, file))
        labels_fail.append(1)

    if file_id in non_fail_ids and len(paths_non_fail) < 30000:
        paths_non_fail.append(os.path.join(DATA_PATH, file))
        labels_non_fail.append(0)

# Exclude data used in autoencoder training
paths_non_fail, labels_non_fail = paths_non_fail[6500:10000], labels_non_fail[6500:10000]
filepaths, labels = [None]*7000, [None]*7000
filepaths[::2], labels[::2] = paths_fail, labels_fail
filepaths[1::2], labels[1::2] = paths_non_fail, labels_non_fail

train_paths, train_labels = filepaths[:6000], labels[:6000]
val_paths, val_labels = filepaths[6000:6500], labels[6000:6500]
test_paths, test_labels = filepaths[6500:], labels[6500:]

### Data Generator

- Create a data generator to read in training, validation and test data by batches
- Use the pretrained autoencoder model to encode the input data
- Model input formats:  
    x -> (n_batch, n_time_steps, n_features)  
    y -> (n_batch, 1)

In [5]:
# Load the pretrained autoencoder model
autoencoder = tf.keras.models.load_model('model_files/lstm_autoencoder')

In [7]:
def data_generator(batch_size, filepaths, labels, length, shift):
    count = 0
    X, y = [], []

    for file, label in zip(filepaths, labels):
        # Load simulation file as numpy array
        arr = joblib.load(os.path.join(DATA_PATH, file))

        # Windowing - create evenly sized samples
        end_index = arr.shape[0] - length

        for i in range(0, end_index, shift):
            y.append(int(label))
            X.append(arr[i:(i+length), 1:])
            count += 1

            if count >= batch_size:
                _X = np.array(X).astype(np.float32)
                enc_X = autoencoder.predict(_X)
                yield (enc_X, np.array(y))
                X, y = [], []
                count = 0

### Model Training and Evaluation
- Train a single layer LSTM model
- Evaluate on test data: accuracy, precision, recall, F1 score

In [10]:
# LSTM
EPOCH_SIZE = 10

# Number of time steps and number of features
n_time_steps = 100
n_feats = 259

model = Sequential()
model.add(LSTM(50, input_shape = (n_time_steps, n_feats)))
model.add(Dense(1, activation = 'sigmoid'))

adam = optimizers.Adam(lr=0.001, clipnorm=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', Precision(), Recall()])
model.summary()

# Start training
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint(
    filepath='checkpoint',
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True
)

model.fit_generator(
    data_generator(256, train_paths, train_labels, 100, 25),
    validation_data=data_generator(256, val_paths, val_labels, 100, 25),
    validation_steps=6,
    epochs=EPOCH_SIZE,
    steps_per_epoch=80,
    callbacks=[early_stop, checkpoint],
    shuffle=True
)

model.save('model')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm_2 (UnifiedLSTM) (None, 50)                62000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 62,051
Trainable params: 62,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


 1/80 [..............................] - ETA: 3:58 - loss: 0.6879 - accuracy: 0.5625 - precision_1: 0.5625 - recall_1: 1.0000

 2/80 [..............................] - ETA: 2:01 - loss: 0.7193 - accuracy: 0.5078 - precision_1: 0.5078 - recall_1: 1.0000

 3/80 [>.............................] - ETA: 1:47 - loss: 0.7136 - accuracy: 0.5156 - precision_1: 0.5156 - recall_1: 1.0000

 4/80 [>.............................] - ETA: 1:46 - loss: 0.7105 - accuracy: 0.5078 - precision_1: 0.5078 - recall_1: 1.0000

...

76/80 [===========================>..] - ETA: 5s - loss: 0.6932 - accuracy: 0.4963 - precision_1: 0.4973 - recall_1: 0.7465

77/80 [===========================>..] - ETA: 4s - loss: 0.6932 - accuracy: 0.4970 - precision_1: 0.4981 - recall_1: 0.7501

78/80 [============================>.] - ETA: 2s - loss: 0.6932 - accuracy: 0.4962 - precision_1: 0.4971 - recall_1: 0.7529

79/80 [============================>.] - ETA: 1s - loss: 0.6932 - accuracy: 0.4970 - precision_1: 0.4982 - recall_1: 0.7564

80/80 [==============================] - 118s 1s/step - loss: 0.6932 - accuracy: 0.4965 - precision_1: 0.4974 - recall_1: 0.7592 - val_loss: 0.6932 - val_accuracy: 0.5000 - val_precision_1: 0.5000 - val_recall_1: 1.0000


In [11]:
test_y, pred_y = [], []

for X, y in data_generator(256, test_paths, test_labels, 100, 25):
    test_y.append(y)
    pred_y.append(model.predict(X))

test_y = np.concatenate(test_y)
pred_y = np.concatenate(pred_y)
pred_y[pred_y >= 0.5] = 1
pred_y[pred_y < 0.5] = 0

# Accuracy, precision, recall, F1 score
acc = accuracy_score(test_y, pred_y)

print('Accuracy on test set: {:.3f}'.format(acc))
print(classification_report(test_y, pred_y))

Accuracy on test set: 0.500
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8956
           1       0.50      1.00      0.67      8964

    accuracy                           0.50     17920
   macro avg       0.25      0.50      0.33     17920
weighted avg       0.25      0.50      0.33     17920



/projects/d19a8e8b-8443-4e4b-86b0-93ceef78058e/.sage/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
